<a href="https://colab.research.google.com/github/RabiyaAkhtar/HurricanDamageAssesment/blob/main/HurricanDamageAssesment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install datasets transformers torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [5]:
!pip install datasets transformers torch torchvision

In [6]:
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
import torch
from torch import nn

# Load the Satellite-Images-of-Hurricane-Damage dataset
dataset = load_dataset("jonathan-roberts1/Satellite-Images-of-Hurricane-Damage")


# Determine the number of unique classes
# Replace 'label' with the name of the column that contains the labels if it's different
num_classes = len(set(dataset['train']['label']))
print(f"Number of classes: {num_classes}")



# Load the image processor and model
processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=num_classes,
    ignore_mismatched_sizes=True  # Allow loading weights with mismatched sizes
)

# Replace the classifier head with a new one
model.classifier = nn.Linear(model.config.hidden_size, num_classes)

print("Model loaded and classifier head adjusted for the dataset's number of classes.")







# Define preprocessing transformations
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=processor.image_mean, std=processor.image_std),
])


from PIL import Image
import requests
from io import BytesIO
import torch

# Function to preprocess dataset
def preprocess_data(examples):
    images = []
    for image_path in examples["image"]:
        try:
            # If the image is a URL, download it
            if isinstance(image_path, str) and image_path.startswith("http"):
                response = requests.get(image_path)
                image = Image.open(BytesIO(response.content)).convert("RGB")
            else:
                # Else it's a local path (assumes image_path is a valid file path)
                image = Image.open(image_path).convert("RGB")

            # Apply transformations
            images.append(transform(image))  # Ensure 'transform' outputs tensors

        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            images.append(torch.zeros(3, 224, 224))  # Append a dummy tensor if loading fails

    labels = examples["label"]
    return {"pixel_values": images, "labels": labels}

# Check the available splits in the dataset
print(dataset)

from datasets import Dataset

# Access the train split
train_dataset = dataset["train"]

# Split the training data into train and validation (80% train, 20% validation)
train_split, validation_split = train_dataset.train_test_split(test_size=0.2).values()

# Preprocess both splits
train_dataset = train_split.map(preprocess_data, batched=True)
validation_dataset = validation_split.map(preprocess_data, batched=True)

# Format datasets for PyTorch
train_dataset.set_format(type="torch", columns=["pixel_values", "labels"])
validation_dataset.set_format(type="torch", columns=["pixel_values", "labels"])






README.md:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

(…)-00000-of-00001-662258fe8a4f018c.parquet:   0%|          | 0.00/27.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Number of classes: 2


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded and classifier head adjusted for the dataset's number of classes.
DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Streaming output truncated to the last 5000 lines.
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71D07FCD30>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71D07FCE50>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71D07FCDC0>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71D07FCD00>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71D07FCC70>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71D07FF880>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71D07FE560>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71D07FFB50>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile ima

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71CEC67970>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71CEC67D90>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71CEC67E20>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71CEC67EB0>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71CEC67F40>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71CEC67FD0>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71CEC65FF0>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71CEC65F60>: read
Error processing image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x128 at 0x7D71CEC65ED0>: read
E

In [7]:
training_args = TrainingArguments(
    output_dir="./results",                # Directory to save checkpoints
    evaluation_strategy="epoch",          # Evaluate at the end of every epoch
    save_strategy="epoch",                # Save model checkpoints every epoch
    learning_rate=5e-5,                   # Learning rate
    per_device_train_batch_size=16,       # Batch size for training
    per_device_eval_batch_size=16,        # Batch size for evaluation
    num_train_epochs=5,                   # Number of epochs
    weight_decay=0.01,                    # Weight decay for optimizer
    logging_dir="./logs",                 # Directory to save logs
    logging_steps=10,                     # Log every 10 steps
    load_best_model_at_end=True,          # Load the best model at the end of training
    metric_for_best_model="accuracy",     # Metric for evaluating the best model
    save_total_limit=2                    # Limit number of checkpoints saved
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00


In [9]:
import evaluate
import torch

# Load the accuracy metric
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return accuracy.compute(predictions=predictions, references=labels)


In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,  # Use validation_dataset instead of test_dataset
    compute_metrics=compute_metrics,
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.714000,0.693174,0.498500
2,0.697000,0.693178,0.498500
3,0.696500,0.693863,0.501500
4,0.693300,0.693545,0.501500


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.714000,0.693174,0.498500
2,0.697000,0.693178,0.498500
3,0.696500,0.693863,0.501500
4,0.693300,0.693545,0.501500
5,0.693100,0.693149,0.498500


TrainOutput(global_step=2500, training_loss=0.6972456060409545, metrics={'train_runtime': 2584.279, 'train_samples_per_second': 15.478, 'train_steps_per_second': 0.967, 'total_flos': 3.09967958458368e+18, 'train_loss': 0.6972456060409545, 'epoch': 5.0})

In [11]:
model.save_pretrained("./hurricane-damage-assessment-model")
processor.save_pretrained("./hurricane-damage-assessment-processor")


['./hurricane-damage-assessment-processor/preprocessor_config.json']

In [ ]:
# Load the model configuration
from transformers import AutoConfig

model_config = AutoConfig.from_pretrained("./hurricane-damage-assessment-model")
print(model_config.id2label)  # This will show the mapping if available


In [23]:
from PIL import Image

# Load fine-tuned model and processor
model = AutoModelForImageClassification.from_pretrained("./hurricane-damage-assessment-model")
processor = AutoImageProcessor.from_pretrained("./hurricane-damage-assessment-processor")

# Process a single image
image = Image.open("hur1.jpg").convert("RGB")


inputs = processor(images=image, return_tensors="pt")

# Perform inference
outputs = model(**inputs)
predicted_class = torch.argmax(outputs.logits, dim=-1).item()

# Define the class labels manually
id2label = {0: "Hurricane Damage", 1: "No Damage"}
label2id = {v: k for k, v in id2label.items()}

# Update the model's configuration with the class labels
model.config.id2label = id2label
model.config.label2id = label2id

# Now use the model to make predictions and print the human-readable label
predicted_class_label = model.config.id2label[predicted_class]
print(f"Predicted class: {predicted_class_label}")

print(f"Predicted class: {predicted_class}")


Predicted class: No Damage
Predicted class: 1


In [20]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [24]:
import gradio as gr
from PIL import Image
import torch
from transformers import AutoModelForImageClassification, AutoImageProcessor

# Load the fine-tuned model and processor
model = AutoModelForImageClassification.from_pretrained("./hurricane-damage-assessment-model")
processor = AutoImageProcessor.from_pretrained("./hurricane-damage-assessment-processor")

# Define the class labels manually
id2label = {0: "Hurricane Damage", 1: "No Damage"}
label2id = {v: k for k, v in id2label.items()}

# Update the model's configuration with the class labels
model.config.id2label = id2label
model.config.label2id = label2id

# Function to make predictions
def predict_damage(image):
    # Process the input image
    inputs = processor(images=image, return_tensors="pt")

    # Perform inference
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=-1).item()

    # Get the human-readable label
    predicted_label = model.config.id2label[predicted_class]

    return predicted_label

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_damage,                        # Prediction function
    inputs=gr.Image(type="pil"),              # Input type: Image
    outputs=gr.Textbox(),                     # Output type: Textbox
    title="Hurricane Damage Assessment",      # Title of the interface
    description="Upload an image of hurricane damage, and the model will classify it as either 'Hurricane Damage' or 'No Damage'.",  # Description
)

# Launch the interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://55b35fb42278687306.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
